
#  高性能プログラミングと性能測定(1) --- OpenMP CPUマルチコアプログラミング


# 1. OpenMP
* OpenMPはCPUでマルチコア並列処理を行うための標準(デファクトスタンダード)
* 新しい仕様ではGPUもサポートしている(どこまでサポートしているかはコンパイラ依存)

* 詳しい仕様が知りたくなったら https://openmp.org/ を参照
  * 最新仕様 https://www.openmp.org/spec-html/5.2/openmp.html
  * 簡潔な文法のリファレンス: https://www.openmp.org/resources/refguides/
* 最小限の覚えるべきキーワード
  * `#pragma omp parallel`
  * `#pragma omp for`
  * `reduction`
* 最小限の覚えるべきAPI関数
```
#include <omp.h> 
```
して
  * omp_get_num_threads();
  * omp_get_thread_num();


# 2. コンパイラ
* この演習環境では, 同じコンパイラでCPUもGPUもサポートしているという理由で, NVIDIA HPC SDKを使う
* ただし, 世の中的にはCPU上のプログラミングといえば gcc, clang (LLVM) などが主流なので, それらも紹介しておく

## 2-1. NVIDIA HPC SDK
* コマンド名:
  * C: `nvc`
  * C++: `nvc++`
* コンパイルオプション:
  * `-mp=multicore` をつけると CPU用のOpenMPがサポートされる
  * `-mp=gpu` をつけると GPU用のOpenMPがサポートされる (次週)
* 上記のコマンドを実行できるようにするために, 以下を実行してください(この環境特有. Wisteriaでは別のやり方(moduleコマンド))

In [ ]:
export PATH=/opt/nvidia/hpc_sdk/Linux_x86_64/24.5/compilers/bin:$PATH

## 2-2. GCC
* コマンド名:
  * C: `gcc`
  * C++: `g++`
* コンパイルオプション:
  * `-fopenmp` をつけると CPU用のOpenMPがサポートされる


# 3. `#pragma omp parallel` 構文
* OpenMPで, 「これがなくては始まらない」プリミティブ
* OpenMPプログラムは1スレッドで実行を開始し(MPIのようにmain関数が複数実行されるのではない), `#pragma omp parallel`に差し掛かると直後に書かれた文を複数のスレッドが実行する
* いくつできるかの規則や制御方法にはいくつかあるが, もっとも基本的なルールは, 実行時の環境変数OMP_NUM_THREADSで指定した数, というもの

* 以下を実行すると, `#pragma omp parallel`の直下に書かれた一文 `printf("in parallel\n");` が複数のスレッドによって実行される

In [ ]:
%%writefile omp_parallel.c
#include <stdio.h>

int main() {
  printf("before parallel\n");
#pragma omp parallel
  printf("in parallel\n");
  printf("after parallel\n");
  return 0;
}


In [ ]:
nvc -fast -mp=multicore omp_parallel.c -o omp_parallel_mp.exe

* `OMP_NUM_THREADS` という環境変数 (「環境変数」を知らない人は以下のようにコマンドの前にある`変数名=値`のことだと思えば良い) を設定すると, `printf("in parallel\n");` (`#pragma omp parallel` 直下の文) を実行するスレッドの数を変えられる
* 以下で`OMP_NUM_THREADS`に設定する値を色々変えて実行してみよ

In [ ]:
OMP_NUM_THREADS=3 ./omp_parallel


* 文法
```
#pragma omp parallel
文
```
* 意味
 * 「文」を複数のスレッドで実行する(典型的には環境変数 `OMP_NUM_THREADS` で指定した数)
  * それらのスレッドを, その文を実行する<font color="blue">チーム</font>と呼ぶ
 * チームの全スレッドが「文」を実行し終えると, `#pragma omp parallel` 全体が実行を終える
 * 再び1スレッドに戻って以降の文を実行する
* 注
 * 複数のスレッドが実行するのは `#pragma omp parallel` の直下に書かれたひとつの文だが, この文自体が複合文 (`{ ... }` で囲まれた複数の文をまとめて一つの文とみなしたもの)だったり, ループだったり, 関数呼び出しを含んで, その関数の中に多数の文を含んでいることがあるので, 実際には複数スレッドで実行される文の数はいくらでも多数であり得る


# 4. `#pragma omp parallel` を実行するスレッド数の指定の仕方色々
* プログラムの中で指定する
  * `#pragma omp parallel` に `num_threads(N)` として `N` 個のスレッドで実行するよう指定できる
  * `N` はCの式でよいのでこれをプログラムの引数で指定したり, プログラム中のデータに応じて調節したりということもやろうと思えばできる
  * ここで指定したものは環境変数 `OMP_NUM_THREADS` による指定よりも優先される
* 以下のプログラムの引数を変えてk実行してみよ

In [ ]:
./omp_parallel_num_threads 3

* 環境変数 `OMP_NUM_THREADS` で指定する (推奨)
  * 上述したとおり
  * 手軽にスレッド数を変更できるので多くの場合はこれを用いる
  * 一般に, プログラムはスレッド数がいくつであっても実行するように書いておき, 実行するときに調節できるようにするのが良いので, その意味でも特に理由がなければこれを使うことを推奨

* 指定しない
  * どちらも指定しなければ (仕様にそう書かれているかは知らないが普通), 環境に備わる仮想コア数を `OMP_NUM_THREADS` に指定したのと同じ効果になる
  * 「仮想コア」については後述
  * 以下のコマンドでそれを知れる

In [ ]:
nproc

* `omp_parallel` を `OMP_NUM_THREADS` 指定せずに実行してみよ

In [ ]:
./omp_parallel

* `OMP_NUM_THREADS` には自由な数を指定してよいが, CPUに搭載されている仮想コア数より大きくしても計算が速くなることはまずない

# 5. omp_num_threads() と omp_thread_num()
*
```
#pragma omp parallel
  S
```
によってSを実行中のスレッドは, 
 * `omp_num_threads()` によってSを実行しているチームのスレッド数
 * `omp_thread_num()` によってその中での自分の番号(スレッド数を$n$として, 0以上$n$未満の数)
を得ることが出来る

* これらを使う場合

```
#include <omp.h>
```

しておく


In [ ]:
%%writefile omp_parallel_rank.c
#include <stdio.h>
#include <omp.h>

int main() {
  printf("hello\n");
#pragma omp parallel
  {
    int omp_nthreads = omp_get_num_threads();
    int omp_rank = omp_get_thread_num();
    printf("world %d/%d\n", omp_rank, omp_nthreads);
  }
  printf("good bye\n");
  return 0;
}

In [ ]:
nvc -fast -mp=multicore omp_parallel_rank.c -o omp_parallel_rank_mp.exe

In [ ]:
OMP_NUM_THREADS=3 ./omp_parallel_rank

In [ ]:
./omp_parallel_rank


# 6. CPU, コア, 仮想コア(ハードウェアスレッド)
* 最近の<font color="blue">CPU</font>はマルチコアCPUで, 1つのチップに複数の, 独立に実行可能な<font color="blue">「コア」</font>が搭載されている
* ひとつのボード(計算ノード)に複数のCPUが搭載されていることもある(<font color="blue">マルチソケット</font>環境)
* さらにひとつのコアにも独立に命令を実行可能な, <font color="blue">「仮想コア(ハードウェアスレッド)」</font>が搭載されている
* 田浦 <a href="https://taura.github.io/cs-alliance/slides/pdf/taura_lecture.pdf" target="_blank">講義スライド</a> pXXを参照
* ソフトウェア(OS)からは仮想コアがひとつのプロセッサとして見える
* ただしハードウェアスレッドは起動時にOFFにすることもでき, その場合は観測されるプロセッサ数は, 物理コア数になる
* OSから見えているプロセッサ数は以下のコマンドで観測できる

In [ ]:
nproc

* CPUの機種名は以下で観測できる

In [ ]:
cat /proc/cpuinfo | head -30


* コアを, 仮想コアと区別して「物理コア」と呼ぶこともある
* 物理コアとか仮想コアの違いはソフトウェアからはほとんど見えないが, 同一の物理コア上の仮想コアは演算器を共有しており, 「一サイクルに実行可能な浮動小数点演算数」みたいな数字は「コア」あたりの数字である. すでに限界性能に近いスレッドを二つ, 同一のコアに置いても性能は倍にならない



* `#pragma omp parallel`はその直下の文を「全員が(重複して)」実行する手段
* 実際に処理を高速化するには一定量の仕事を複数のスレッドで「分割」して処理する必要がある
* そのための手段が work sharing 構文. そのひとつが `#pragma omp for`


# 7. `#pragma omp for` 構文
* `#pragma omp parallel`で指定された文を実行中にスレッドが `#pragma omp for` に到達すると, その直下に書かれた文 (for 文でなくてはならない)の繰り返しをスレッド間で分け合って実行する
* 仕事をスレッド間で分け合って実行するので, work sharing構文と呼ぶ 
* OpenMPには他のwork sharing構文もあるがこの演習ではそれらはやらない


* 以下は `#pragma omp for` の効果を確かめる簡単な例

In [ ]:
%%writefile omp_parallel_for.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

int main(int argc, char ** argv) {
  int m = (1 < argc ? atoi(argv[1]) : 10);
  printf("before parallel\n");
#pragma omp parallel
  {
    printf("in parallel, before for\n");
#pragma omp for
    for (int i = 0; i < m; i++) {
      printf("i = %3d\n", i);
    }
    printf("in parallel, after for\n");
  }
  printf("after parallel\n");
  return 0;
}


In [ ]:
nvc -fast -mp=multicore omp_parallel_for.c -o omp_parallel_for_mp.exe

In [ ]:
OMP_NUM_THREADS=3 ./omp_parallel_for 10

* `m`回の繰り返しが複数のスレッドで分割されて実行されるため, 表示される順番が i の小さい順番とは限らなくなることに注意

* また, `#pragma omp for` を取り除くと, ループのすべての繰り返しがすべてのスレッドで実行されることに注意 (やってみよ)
* 普通はこんなプログラムは書かないが, 仕様を理解するための実験としてやってみよ

* OpenMPがやってくれることは基本的には for文の繰り返しを複数のスレッドで分割して実行するということである
* 逆に言うと, 計算時間の多くを並列に実行可能なfor文が占めていることが, OpenMPによる並列化が成功するための条件である



# 8. `schedule` 句
* `#pragma omp for` では, 繰り返しをどのようにスレッドに割り当てるかを, `schedule` 句によって指定できる
* 繰り返し回数が$M$回, スレッド数が$T$個としたときに, 
  * `#pragma omp for schedule(static)` : はじめの$\lceil M/T \rceil$回をスレッド0, 次の$\lceil M/T \rceil$回をスレッド1, ... が実行 ($M$が$T$で割り切れなければ最後の方に少ないスレッドが出る場合も有る)
  * `#pragma omp for schedule(static, c)` : はじめの$c$回をスレッド0, 次の$c$回をスレッド1, ... が実行 (最後まで到達したらまたスレッド0に戻る)
  * `#pragma omp for schedule(dynamic)` : 各スレッドが, 「まだ実行されていない繰り返しを1つ実行する」を繰り返す. どの繰り返しをどのスレッドが実行することになるかは予測できない. 
  * `#pragma omp for schedule(dynamic, c)` : `dynamic`と同じだが, 繰り返しは一度につき$c$個ずつまとめてわりあてる. つまり「まだ実行されていない繰り返しを$c$個 (残った繰り返し数が$c$未満の場合はすべて) 実行する」を繰り返す. 
  * `#pragma omp for schedule(guided)` : `dynamic`と似ているが最初のうちは一度につき, 多めの繰り返しを割当て, 徐々に少なくしていく
  * `#pragma omp for schedule(guided, c)` : `guided`と似ているが最低でも$c$個割り当てる
  * `#pragma omp for schedule(runtime)` : 実行する際に環境変数 `OMP_SCHEDULE` で, 以下のようにschedule句を指定できる.
```
OMP_SCHEDULE=static コマンド
```  
* 何も書かなかった時のデフォルトが仕様で定められているかどうかは知らないが, 普通は初めの`static`の動作をする

* 以下のプログラムは, 各繰り返しを`usleep(i * 100000)`によって約$i \times 100$ミリ秒 ($=i \times 0.1) 秒かかるようにした上で, どの繰り返しをどのスレッドが実行したかを表示するもの
* `schedule`句を書き換えてコンパイル, 実行し, 上記を確認せよ
* どのようなときに`static` / `dynamic` を用いるべきか?


In [ ]:
%%writefile omp_parallel_for_thread_num.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>
#include <unistd.h>

int main(int argc, char ** argv) {
  int m = (1 < argc ? atoi(argv[1]) : 10);
  printf("before parallel\n");
#pragma omp parallel
  {
    int nt = omp_get_num_threads();
#pragma omp for schedule(static)
    for (int i = 0; i < m; i++) {
      int t = omp_get_thread_num();
      usleep(i * 100000);
      printf("i = %3d, by thread %3d of %3d\n", i, t, nt);
    }
  }
  printf("after parallel\n");
  return 0;
}


In [ ]:
nvc -fast -mp=multicore omp_parallel_for_thread_num.c -o omp_parallel_for_thread_num_mp.exe

In [ ]:
OMP_NUM_THREADS=2 ./omp_parallel_for_thread_num 10

* 何度も書き換えてコンパイルするのが面倒になったら`schedule(runtime)` を使って, `OMP_SCHEDULE`をコマンドラインで指定するのが吉

In [ ]:
OMP_NUM_THREADS=2 OMP_SCHEDULE=static ./omp_parallel_for_thread_num 10


# 9. 台数効果の目撃
* $T$個のスレッドを用いても速度が$T$倍(近く)にならないことはしばしば
* そもそも$T > $コア数のときは複数のスレッドが同じコア上で動くことになり, コア数以降の向上は望めない(上のプログラムのようにスレッドがわざと(usleepで)休眠していたり, ファイルのIOなどで長時間止まっている場合は別)
* OpenMPをそのような設定で使うことはあまりない
* $T \leq $コア数であっても, 速度が$T$倍 (実行時間が$1/T$倍) とは程遠くなる理由がいくつも有る
  1. 1つのスレッドによって実行される処理が小さすぎる場合 ($T$個スレッドが実行を開始して, 全員が終了するのを待つ, というオーバーヘッドが目立つ)
  1. OSが速やかに$T$個のスレッドを別々のコアに割り当ててくれないことがある
  1. スレッド間でデータを共有している場合にデータアクセスのコストが1スレッドの場合に比べて大きくなる(詳細は計算機の仕組みに関わるのでここでは深入りしない)
* 3番目の問題はアルゴリズムの本質に関わる現象で容易に除去できない場合が多いが,
  * 1についてはそういうものだと思っておく(あまり短すぎる処理を複数コアで性能向上はできない. あまりそうする意味もないのでやる必要もない)
  * 2については, 実行時に`OMP_PROC_BIND=true` と環境変数を設定することで改善することがある. これは各スレッドを特定のコアでしか実行しないようにOSに指示するという効果があるため, 2の問題を緩和できる

* 以下では, 計算自身にあまり意味はないが簡単な例題で性能向上を目撃してみる
* なお `#pragma omp parallel for` は `#pragma omp parallel` の直後に `#pragma omp for` を書いたのと同じ効果を持つ

In [ ]:
%%writefile omp_speedup.c
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>
#include <time.h>
#include <unistd.h>

/* x = ax + b をひたすら n 回繰り返す.
   (|a| < 1.0 なら c によらず, x = b / (1 - a) に収束).
   n 回 mul + add を行う (-> 2 n flops) */
double lin_rec(double a, double b, double c, long n) {
  double x = c;
  for (long j = 0; j < n; j++) {
    x = a * x + b;
  }
  return x;
}

int main(int argc, char ** argv) {
  long m = (1 < argc ? atoi(argv[1]) : 10);
  long n = (2 < argc ? atoi(argv[2]) : 1000 * 1000);
  double * x = (double *)calloc(sizeof(double), m);
  assert(x);
  printf("m = %ld, n = %ld\n", m, n);
  /* 計測開始 */
  double t0 = omp_get_wtime();
#pragma omp parallel for
  for (long i = 0; i < m; i++) {
    x[i] = lin_rec(0.99, i + 1, 1.0, n);
  }
  /* 計測終了 */
  double t1 = omp_get_wtime();
  double dt = t1 - t0;          /* sec */
  /* 答え表示 (x[i] = 100 * (i + 1) くらいのはず) */
  for (long i = 0; i < m; i++) {
    printf("x[%3ld] = %9.3f\n", i, x[i]);
  }
  double flops = 2. * (double)m * (double)n;
  printf("elapsed    : %7.3f  sec\n", dt);
  printf("elapsed/m  : %7.3f msec\n", dt / m * 1e3);
  printf("elapsed/n  : %7.3f nsec\n", dt / n * 1e9);
  printf("elapsed/mn : %7.3f nsec\n", dt / (m * n) * 1e9);
  printf("flops      : %.2e\n", flops);
  printf("%.3f GFLOPS\n", flops / dt * 1e-9);
  return 0;
}


In [ ]:
nvc -fast -mp=multicore omp_speedup.c -o omp_speedup_mp.exe

* 以下は, $m = 72$, $n = 100 \times 1000 \times 1000$として実行する
* スレッド数を変えて, 仮想コア数付近までの性能向上 (GFLOPS値の向上), それ以降の頭打ちを確認せよ

In [ ]:
OMP_NUM_THREADS=1 ./omp_speedup_mp.exe 72 $((100 * 1000 * 1000))

* 手動でやるのが嫌になったら以下で一撃で実行

In [ ]:
for x in 1 2 3 4 6 8 12 16 20 24 28 32; do
    echo -n "$x "
    OMP_NUM_THREADS=${x} OMP_PROC_BIND=true ./omp_speedup_mp.exe 72 $((100 * 1000 * 1000)) | grep GFLOPS
done

* 結果を以下で可視化 (上の結果をコピペせよ)

In [ ]:
import matplotlib.pyplot as plt

DATA = r"""
1 xxxx GFLOPS
2 xxxx GFLOPS
3 xxxx GFLOPS
4 xxxx GFLOPS
6 xxxx GFLOPS
8 xxxx GFLOPS
12 xxxx GFLOPS
16 xxxx GFLOPS
20 xxxx GFLOPS
24 xxxx GFLOPS
28 xxxx GFLOPS
32 xxxx GFLOPS
"""

def main():
    data = DATA.strip().split("\n")
    X = []
    Y = []
    for line in data:
        fields = line.strip().split()
        if len(fields) != 3:
            continue
        x, y = fields[:2]
        X.append(float(x))
        Y.append(3.0)
    plt.ylabel("GFLOPS")
    plt.xlabel("num_threads")
    plt.plot(X, Y)
    plt.show()

main()


# 10. OpenMPにおけるデータの共有
* OpenMPのスレッドは基本的に全てのデータ(変数や配列)を<font color="blue">「共有」</font>している
* 「共有」しているとは, 大雑把にいえば, どのスレッドが変数に書き込んだ値も, 他のスレッドに見えるということである
* そのことはこれまでの例題プログラムでも暗黙的に前提としていたことで, 例えば以下で, 
```
#pragma omp parallel for
  for (long i = 0; i < m; i++) {
    x[i] = lin_rec(0.99, i + 1, 1.0, n);
  }
  /* 計測終了 */
  double t1 = omp_get_wtime();
  double dt = t1 - t0;          /* sec */
  /* 答え表示 (x[i] = 100 * (i + 1) くらいのはず) */
  for (long i = 0; i < m; i++) {
    printf("x[%3ld] = %9.3f\n", i, x[i]);
  }
```
各スレッドが`x[i]`に書き込んだ値が「答え表示」で表示できるのも, スレッドがデータを共有しているからである

* データが共有されているということは便利でもある一方で実は気をつけなくてはいけないことがある
* 以下では, データが共有されていることで生ずる問題<font color="red">競合状態</font>とその解消法について説明する



# 11. 競合状態
* 以下のコードをOpenMPで並列化することを考える


In [ ]:
%%writefile integral.c
#include <stdio.h>
#include <stdlib.h>

double int_inv_1_x2(double a, double b, long n) {
  double s = 0.0;
  double dx = (b - a) / (double)n;
  for (long i = 0; i < n; i++) {
    double x = a + i * dx;
    s += 1 / (1 + x * x);
  }
  return s * dx;
}

int main(int argc, char ** argv) {
  double a = (argc > 1 ? atof(argv[1]) : 0.0);
  double b = (argc > 2 ? atof(argv[2]) : 1.0);
  long n   = (argc > 3 ? atof(argv[3]) : 1000L * 1000L * 1000L);
  double s = int_inv_1_x2(a, b, n);
  printf("s = %f\n", s);
  return 0;
}

In [ ]:
nvc -fast -mp=multicore integral.c -o integral_mp.exe


* ここにそのまま pragma parallel, pragma for を当てはめると以下のようになる


In [ ]:
%%writefile omp_integral_racy.c
#include <stdio.h>
#include <stdlib.h>

double int_inv_1_x2(double a, double b, long n) {
  double s = 0.0;
  double dx = (b - a) / (double)n;
#pragma omp parallel for
  for (long i = 0; i < n; i++) {
    double x = a + i * dx;
    s += 1 / (1 + x * x);
  }
  return s * dx;
}

int main(int argc, char ** argv) {
  double a = (argc > 1 ? atof(argv[1]) : 0.0);
  double b = (argc > 2 ? atof(argv[2]) : 1.0);
  long n   = (argc > 3 ? atof(argv[3]) : 1000L * 1000L * 1000L);
  double s = int_inv_1_x2(a, b, n);
  printf("s = %f\n", s);
  return 0;
}

In [ ]:
nvc -fast -mp=multicore omp_integral_racy.c -o omp_integral_racy_mp.exe


* 正解は $\pi/4$で 1スレッドで実行すれば問題なく正解が出る


In [ ]:
OMP_NUM_THREADS=1 time ./omp_integral_racy_mp.exe


* 2スレッド以上だと正解は出ない上, 毎回答えも違う(非決定的な挙動)


In [ ]:
OMP_NUM_THREADS=4 time ./omp_integral_racy_mp.exe


* その理由を考察する(講義スライド pXX 〜に答えがある)

<pre>
#pragma omp parallel for
  for (long i = 0; i < n; i++) {
    double x = a + i * dx;
    s += 1 / (1 + x * x);
  }
</pre>

において, 変数 sは全部のスレッドで共有されている(注: `x` は `#pragma omp parallel` で実行される文の内部で定義されており, そのような変数はスレッドごとに別の変数になる. つまり共有されない)
* 複数のスレッドが同じ変数`s`を更新することになる
* よくないことが起きる具体的なシナリオはスライドpXX を参照. `s += ...` は実際には, `s` を読み出し, `...` を足して, それをまた`s`に書き戻す. 読み出しから書き戻しまでの間に別のスレッドが `s` を更新するとおかしなことになる



* このように, 複数のスレッドが同じ変数を使っていて, 少なくとも一人は更新している状況を<font color="red">「競合状態」</font>と呼び, 大概のプログラムは意図した動作をしない
* 競合状態があったら必ず何か手を打たないといけないと思っておくべき



## 11-1. reduction 節を用いた解決法
* 今回の変数 `s` に対する足し込みのような演算(もう少し一般的な話は後で述べる)に対してはとりわけ簡単な解決法
* `#omp pragma parallel` もしくは `#omp pragma for` の中に `reduction(演算:変数)` という節を付け加えれば良い


In [ ]:
%%writefile omp_integral_reduction.c
#include <stdio.h>
#include <stdlib.h>

double int_inv_1_x2(double a, double b, long n) {
  double s = 0.0;
  double dx = (b - a) / (double)n;
#pragma omp parallel for reduction(+:s)
  for (long i = 0; i < n; i++) {
    double x = a + i * dx;
    s += 1 / (1 + x * x);
  }
  return s * dx;
}

int main(int argc, char ** argv) {
  double a = (argc > 1 ? atof(argv[1]) : 0.0);
  double b = (argc > 2 ? atof(argv[2]) : 1.0);
  long n   = (argc > 3 ? atof(argv[3]) : 1000L * 1000L * 1000L);
  double s = int_inv_1_x2(a, b, n);
  printf("s = %f\n", s);
  return 0;
}

In [ ]:
nvc -fast -mp=multicore omp_integral_reduction.c -o omp_integral_reduction_mp.exe

In [ ]:
time ./omp_integral_reduction_mp.exe


* reductionは一般に多数の値を縮約する演算 o ($s = s_0 {\rm o} s_1 {\rm o}  ... {\rm o} s_{n-1}$)であって, 適用する順番を変えても同じ答えが出る場合に使われる
* つまり足し算(+)であれば $s_0 + s_1 + s_2 + s_3$を, 
$$(((s_0 + s_1) + s_2) + s_3)$$
と計算しても良いし 
$$(s_0 + s_1) + (s_2 + s_3)$$
と計算しても良い. 
* 見ての通り後者のように計算すれば $(s_0 + s_1)$ と $(s_2 + s_3)$ を並行に計算できることになる
* したがって reduction は, 結合則と交換則が成り立つような演算を多数の要素に施す場合に使え, 代表例としては, +, *, max, min, などがある

* ただし実は, 浮動小数点数(`double`, `float`など)の足し算には丸め誤差が生ずる
* そのため浮動小数点数を多数足し合わせる場合, その順番により結果が変わりうる
* `reduction`を使った場合, 1スレッドの実行と答えがずれる, スレッド数によって答えが少し変わるということは普通に生ずる



# 12. 終わりに一言
* これでOpenMPを用いて, マルチコア(複数のコアを使った)並列処理ができるようになった
* ただしこれだけでCPUの性能をどのくらい引き出せるかと言うと, 本来性能には程遠い
* その理由は主にSIMD命令を用いていないこと, 命令間の依存関係によって, 命令レベル並列を引き出せていないこと, である
